In [1]:
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tqdm.notebook as tqdm
import numpy as np
import yolov5
from yolov5.utils.loss import compute_loss

In [2]:
class LisaDataset(torch.utils.data.IterableDataset):

    def __init__(self, verbose=False, output_width_height=True):
        df1 = pd.read_csv('../data/lisa/Annotations/Annotations/daySequence1/frameAnnotationsBOX.csv', sep=';')
        df2 = pd.read_csv('../data/lisa/Annotations/Annotations/daySequence2/frameAnnotationsBOX.csv', sep=';')
        self.df = df1.append(df2).reset_index(drop=True)
        self.verbose = verbose
        self.iternum = 0
        self.output_width_height = output_width_height
        
    def show_bboxes(self, im, bboxes_tensor):
        fig, ax = plt.subplots()
        ax.imshow(im)
        for bbox in bboxes_tensor:
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

    def __len__(self):
        return len(self.df)

    def __iter__(self):
        for i in range(len(self.df)):
            fn = self.df.iloc[i]["Filename"]
            ds = 'daySequence1' if 'daySequence1' in fn else 'daySequence2'
            img = cv2.imread(f'../data/lisa/{ds}/{ds}/frames/{fn.split("/")[1]}')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            bboxes_tensor = self.df[self.df['Filename'] == fn].drop([c for c in self.df.columns if 'corner' not in c], axis=1).values
            for bbox in bboxes_tensor:
                if self.verbose:
                    self.show_bboxes(img, bbox)
                if self.output_width_height:
                    bbox_corrected = bbox.copy()
                    bbox_corrected[2] -= bbox_corrected[0]
                    bbox_corrected[3] -= bbox_corrected[1]
                sample = {'image': img, 'bbox': bbox_corrected} 
                self.iternum += 1
                yield sample

In [ ]:
import yaml

train_data_loader = torch.utils.data.DataLoader(LisaDataset(verbose=False), batch_size=10, shuffle=False)
test_data_loader = torch.utils.data.DataLoader(LisaDataset(verbose=False), batch_size=10, shuffle=False)

with open('../utils/yolov5l.yml') as f:
    conf = yaml.load(f, Loader=yaml.SafeLoader)
    
model = torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True, autoshape=False)

with open('../utils/hyp.finetune.yaml') as f:
    hyp = yaml.load(f, Loader=yaml.SafeLoader)

model.hyp = hyp
model.nc = conf['nc']

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

for epoch in range(10):
    for data in data_loader:
        imgs = data['image'] / 255.
        imgs = imgs.permute(0,3,1,2)
        true = data['bbox']
        true_yolo = torch.ones(true.shape[0], true.shape[1]+2)*9
        true_yolo[:, 2:] = true
        true_yolo[:,0] = torch.tensor(list(range(true_yolo.shape[0])))
        p = model(imgs)
        loss, _ = compute_loss(p, true_yolo, model)
        print(loss.item())
        loss.backward()
        optimizer.step()

Using cache found in /Users/dmevdok/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      7040  models.common.Focus                     [3, 64, 3]                    
  1                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  2                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  3                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  4                -1  1   1611264  models.common.C3                        [256, 256, 9]                 
  5                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  6                -1  1   6433792  models.common.C3                        [512, 512, 9]                 
  7                -1  1   4720640  models.common.Conv          

0.05748751759529114
0.0414251908659935
0.034757211804389954
0.02306497097015381
0.016733109951019287
0.012382004410028458
0.008641124702990055
0.006898534018546343
0.005734425038099289
0.004534943029284477
0.0035442167427390814
0.0029728831723332405
0.0030786555726081133
0.0026255599223077297
0.002480969997122884
0.0023973064962774515
0.002429330488666892
0.002405565232038498
0.002289490308612585
0.0024537835270166397
0.0020122439600527287
0.0019375423435121775
0.001689899479970336
0.0015902529703453183
0.0014072675257921219
0.0012219368945807219
0.0011172706726938486
0.0009987213416025043
0.0009518571896478534
0.0008976320386864245
0.0009112157858908176


In [164]:
p = model(imgs)

In [161]:
from yolov5.utils.loss import build_targets, compute_loss

(tensor([0.01626], grad_fn=<MulBackward0>),
 tensor([0.00000, 0.00542, 0.00000, 0.00542]))

Using cache found in /Users/dmevdok/.cache/torch/hub/ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      7040  models.common.Focus                     [3, 64, 3]                    
  1                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  2                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  3                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  4                -1  1   1611264  models.common.C3                        [256, 256, 9]                 
  5                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  6                -1  1   6433792  models.common.C3                        [512, 512, 9]                 
  7                -1  1   4720640  models.common.Conv          

Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
   

In [60]:
compute_loss(p)

TypeError: compute_loss() missing 2 required positional arguments: 'targets' and 'model'